In [30]:
import pandas as pd
pd.set_option('display.max_columns', 300)
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols
from datetime import datetime
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn import metrics
import pickle
try:
    from sklearn.preprocessing import OrdinalEncoder # just to raise an ImportError if Scikit-Learn < 0.20
    from sklearn.preprocessing import OneHotEncoder
except ImportError:
    from future_encoders import OneHotEncoder # Scikit-Learn < 0.20

## Step 1: Read in hold out data, scalers, and best model

In [31]:
holdout = pd.read_csv('kc_house_data_test_features.csv', index_col=0)





holdout.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [32]:
final_scaler = pd.read_pickle('house_scaler_final2.pickle')
final_model = pd.read_pickle('house_model_final2.pickle')

## Step 2: Feature Engineering for holdout set

Remember we have to perform the same transformations on our holdout data (feature engineering, extreme values, and scaling) that we performed on the original data.

Change format to date

In [33]:
holdout.reset_index(inplace=True)
holdout=holdout[['id', 'date', 'bedrooms', 'bathrooms',
        'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition',
        'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
        'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']]
holdout.set_index('id', inplace=True)
holdout['date']= pd.to_datetime(holdout['date'])
holdout.head()




,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,
1974300020,2014-08-27,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1974300020,2015-02-18,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
3630020380,2014-11-07,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
1771000290,2014-12-03,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
5126310470,2015-01-15,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


make condition categorical

In [34]:
holdout.condition=holdout.condition.map(lambda x: str(x))
housing_no_cond = holdout.drop('condition', axis=1)
housing_cond = holdout[['condition']]
cat_encoder = OneHotEncoder()
housing_cond_1hot = cat_encoder.fit_transform(housing_cond)
housing_cond_1hot.toarray()
X = np.c_[(housing_no_cond, housing_cond_1hot.toarray())]
cols = housing_no_cond.columns.tolist() +cat_encoder.categories_[0].tolist()
holdout = pd.DataFrame(X, columns=cols)
holdout.head()

/Users/benspilsbury/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,1.0,2.0,3.0,4.0,5.0
0,2014-08-27,4,2.5,2270,11500,1,0,0,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,0,0,1,0,0
1,2015-02-18,4,2.5,2270,11500,1,0,0,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,0,0,1,0,0
2,2014-11-07,3,2.5,1470,1779,2,0,0,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576,0,0,1,0,0
3,2014-12-03,3,1.75,1280,16200,1,0,0,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565,0,0,1,0,0
4,2015-01-15,4,2.75,2830,8126,2,0,0,8,2830,0,2005,0,98059,47.4863,-122.14,2830,7916,0,0,1,0,0


make categorical zipcode

In [35]:
housing_no_zip = holdout.drop('zipcode', axis=1)
housing_zip = holdout[['zipcode']]
zip_encoder = OneHotEncoder()
housing_zip_1hot = zip_encoder.fit_transform(housing_zip)
housing_zip_1hot.toarray()
X = np.c_[(housing_no_zip, housing_zip_1hot.toarray())]
cols = housing_no_zip.columns.tolist() +zip_encoder.categories_[0].tolist()
holdout = pd.DataFrame(X, columns=cols)
#holdout.columns
column_fix=[         'date',               'bedrooms',     'bathrooms',
         'sqft_living',      'sqft_lot',        'floors',    'waterfront',
                'view',         'grade',    'sqft_above', 'sqft_basement',
            'yr_built',  'yr_renovated',           'lat',          'long',
       'sqft_living15',    'sqft_lot15',             'condition_1',             'condition_2',
                   'condition_3',             'condition_4',             'condition_5',         'zip_98001',
               'zip_98002',         'zip_98003',         'zip_98004',         'zip_98005',
               'zip_98006',         'zip_98007',         'zip_98008',         'zip_98010',
               'zip_98011',         'zip_98014',         'zip_98019',         'zip_98022',
               'zip_98023',         'zip_98024',         'zip_98027',         'zip_98028',
               'zip_98029',         'zip_98030',         'zip_98031',         'zip_98032',
               'zip_98033',         'zip_98034',         'zip_98038',         'zip_98039',
               'zip_98040',         'zip_98042',         'zip_98045',         'zip_98052',
               'zip_98053',         'zip_98055',         'zip_98056',         'zip_98058',
               'zip_98059',         'zip_98065',         'zip_98070',         'zip_98072',
               'zip_98074',         'zip_98075',         'zip_98077',         'zip_98092',
               'zip_98102',         'zip_98103',         'zip_98105',         'zip_98106',
               'zip_98107',         'zip_98108',         'zip_98109',         'zip_98112',
               'zip_98115',         'zip_98116',         'zip_98117',         'zip_98118',
               'zip_98119',         'zip_98122',         'zip_98125',         'zip_98126',
               'zip_98133',         'zip_98136',         'zip_98144',         'zip_98146',
               'zip_98148',         'zip_98155',         'zip_98166',         'zip_98168',
               'zip_98177',         'zip_98178',         'zip_98188',         'zip_98198',
               'zip_98199']
holdout.columns=column_fix

/Users/benspilsbury/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [36]:
month_dict={'1':"Jan", '2':'Feb', '3':'Mar', '4':'Apr', '5':'May', '6':'Jun', '7':'Jul',
           '8':'Aug', '9':'Sep', '10':'Oct', '11':'Nov', '12':'Dec'}
holdout['month']=holdout.date.map(lambda x: month_dict[str(x.month)])

housing_no_month = holdout.drop('month', axis=1)
housing_month = holdout[['month']]

month_encoder = OneHotEncoder()
housing_month_1hot = month_encoder.fit_transform(housing_month)
housing_month_1hot.toarray()
X = np.c_[(housing_no_month, housing_month_1hot.toarray())]
cols = housing_no_month.columns.tolist() +month_encoder.categories_[0].tolist()
holdout = pd.DataFrame(X, columns=cols)

In [37]:
holdout["build_reno_year"]= np.where(holdout['yr_renovated'] == 0, holdout['yr_built'], holdout['yr_renovated'])

In [38]:
holdout['sqft_living_sq']=holdout.sqft_living.map(lambda x: x**2)
holdout['sqft_lot_sq']=holdout.sqft_lot.map(lambda x: x**2)

In [39]:
holdout["seattle"]= np.where(holdout['long'] < -122, 1, 0)

In [40]:
holdout["sqft_living_urban"]=holdout.sqft_living*holdout.seattle
holdout["sqft_lot_urban"]=holdout.sqft_lot*holdout.seattle

In [41]:
holdout=holdout.drop(columns=['date','yr_renovated', 'sqft_lot15', 'zip_98112', 'sqft_lot_sq'])

In [42]:
holdout = holdout.apply(pd.to_numeric)
holdout.info()
holdout.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Columns: 105 entries, bedrooms to sqft_lot_urban
dtypes: float64(90), int64(15)
memory usage: 3.5 MB


,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,grade,sqft_above,sqft_basement,yr_built,lat,long,sqft_living15,condition_1,condition_2,condition_3,condition_4,condition_5,zip_98001,zip_98002,zip_98003,zip_98004,zip_98005,zip_98006,zip_98007,zip_98008,zip_98010,zip_98011,zip_98014,zip_98019,zip_98022,zip_98023,zip_98024,zip_98027,zip_98028,zip_98029,zip_98030,zip_98031,zip_98032,zip_98033,zip_98034,zip_98038,zip_98039,zip_98040,zip_98042,zip_98045,zip_98052,zip_98053,zip_98055,zip_98056,zip_98058,zip_98059,zip_98065,zip_98070,zip_98072,zip_98074,zip_98075,zip_98077,zip_98092,zip_98102,zip_98103,zip_98105,zip_98106,zip_98107,zip_98108,zip_98109,zip_98115,zip_98116,zip_98117,zip_98118,zip_98119,zip_98122,zip_98125,zip_98126,zip_98133,zip_98136,zip_98144,zip_98146,zip_98148,zip_98155,zip_98166,zip_98168,zip_98177,zip_98178,zip_98188,zip_98198,zip_98199,Apr,Aug,Dec,Feb,Jan,Jul,Jun,Mar,May,Nov,Oct,Sep,build_reno_year,sqft_living_sq,seattle,sqft_living_urban,sqft_lot_urban
0,4,2.50,2270,11500,1.0,0,0,8,1540,730,1967,47.7089,-122.241,2020,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1967,5152900,1,2270,11500
1,4,2.50,2270,11500,1.0,0,0,8,1540,730,1967,47.7089,-122.241,2020,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1967,5152900,1,2270,11500
2,3,2.50,1470,1779,2.0,0,0,8,1160,310,2005,47.5472,-121.998,1470,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2005,2160900,0,0,0
3,3,1.75,1280,16200,1.0,0,0,8,1030,250,1976,47.7427,-122.071,1160,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1976,1638400,1,1280,16200
4,4,2.75,2830,8126,2.0,0,0,8,2830,0,2005,47.4863,-122.140,2830,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2005,8008900,1,2830,8126


In [43]:
transformed_holdout = final_scaler.transform(holdout)



## Step 3: Predict the holdout set

In [47]:
final_answers = final_model.predict(transformed_holdout)

array([547207.81505196, 552529.81505196, 437725.12799557, ...,
       361253.38407069, 413054.22450606, 359194.02610437])

## Step 4: Export your predictions

In [49]:
pd.DataFrame(final_answers).to_csv('housing_preds_ben_spilsbury.csv')

#pd.DataFrame(np_array).to_csv("path/to/file.csv")